In [21]:
using Pkg
using CSV
using Distributions
using DataFrames
using Dates
using Plots
using Random
using LinearAlgebra
using LaTeXStrings
using Lasso
using Statistics
using GLMNet

### 1 Loading the data

In [22]:
# Change the working directory
cd("//your_directory")

# Load the necessary packages
using CSV, DataFrames

# Read the data
data = CSV.read("wage2015_subsample_inference.csv", DataFrame,
                types = Dict(:occ2 => String, :ind2 => String));

# Separate the features and the target variable
data = select(data, Not(["wage", "rownames"])); # Drop columns 'wage' and 'lwage'

In [23]:
# Create the design matrix

design =@formula(lwage ~ 1 + sex + (exp1 + exp2 + exp3 + exp4) * (hsg + scl + clg + ad + so + we + ne + occ2 + ind2) + (hsg + scl + clg + ad) * (so + we + ne + occ2 + ind2) + (so + we + ne) * (occ2 + ind2) + occ2 * ind2);
X_flexible = modelmatrix(design, data);
X_flexible = X_flexible[:,2:end];

In [24]:
y = [data[:,1];;]

5150×1 Matrix{Float64}:
 2.2633643798407643
 3.872802292274865
 2.403126322215923
 2.634927936273247
 3.361976668508874
 2.4622152385859297
 2.9565115604007097
 2.9565115604007097
 2.4849066497880004
 2.9565115604007097
 ⋮
 3.117779707996832
 2.822980167776187
 3.1796551117149194
 2.6280074934286737
 2.6925460145662448
 3.138833117194664
 3.649658740960655
 3.4955080611333966
 2.8511510447428834

### 2 Creating the Lasso Cross-Validation Procedure

4. The `log_grid` function is pretty straight forward

In [25]:
function log_grid(lower::Int, upper::Int, log_step::Float64)
    log_grid = range(lower, stop=upper, length= Int(1 /log_step))
    return exp.(log_grid)
end

log_grid (generic function with 1 method)

5. To code the `k_folds` function, there are many different approaches. We exploited the kronecker product operation and block matrices to build the $k$-folds. Also, we addressed the issue of divisibility between the sample size $n$ and $k$ using an if-else statement depending on the module of $n/k$

In [26]:
function k_folds(data::AbstractArray, k::Int = 5)
    mdl = size(data, 1) % k
    floor = size(data, 1) ÷ k 

    if mdl == 0
        trues = fill(1, floor, 1)
        split_matrix = kron(I(k), trues)
    else
        trues_g1 = fill(1, floor + 1, 1)
        split_matrix_g1 = kron(I(mdl), trues_g1)
        
        trues_g2 = fill(1, floor, 1)
        split_matrix_g2 = kron(I(k - mdl), trues_g2)
        
        split_matrix = [split_matrix_g1  zeros(size(split_matrix_g1, 1), size(split_matrix_g2, 2));
                        zeros(size(split_matrix_g2, 1), size(split_matrix_g1, 2))  split_matrix_g2]
    end
    
    sm_bool = split_matrix .== 1
    splits = [sm_bool[:, x] for x in 1:k]
    
    return splits
end

k_folds (generic function with 2 methods)

6. For the `optimal_lambda` search function, we basically adapted the code provided in the labs so it can use the functions of log-grid and our own $k$-folds function

In [27]:
using GLMNet
function optimal_lambda(Y::AbstractVector, X::AbstractArray, lambda_bounds::Tuple{Int, Int}, k::Int = 5; niter::Int = 100)
    Y = vec(Y) 

    if ndims(X) == 1
        X = reshape(X, :, 1)
    end

    folds = k_folds(X, k)
    all_lambdas = log_grid(lambda_bounds[1],lambda_bounds[2], 1/niter)
    all_mse = zeros(niter)

    for (j, l) in enumerate(all_lambdas)
        split_pes = zeros(k)
        
        for i in 1:k
            X_train = X[.!folds[i], :]
            X_test = X[folds[i], :]
            y_train = Y[.!folds[i]]
            y_test = Y[folds[i]]

            model = glmnet(X_train, y_train, alpha=1.0, lambda=[l])
            predict = GLMNet.predict(model, X_test)

            pe = sum((y_test - predict).^2)
            split_pes[i] = pe
        end

        all_mse[j] = mean(split_pes)
    end

    selected = argmin(all_mse)
    optimal_lambda = all_lambdas[selected]
    optimal_model = glmnet(X, Y, alpha=1.0, lambda=[optimal_lambda])
    optimal_coef = [optimal_model.a0;optimal_model.betas[:]]

    output = Dict(
        "optimal_lambda" => optimal_lambda,
        "optimal_coef" => optimal_coef,
        "all_lambdas" => all_lambdas,
        "all_mse" => all_mse
    )

    return output
end

optimal_lambda (generic function with 2 methods)

7. The `predict_model` function can be easily implemented using the results of `optimal_function`

In [28]:
function predict_model(optimal_model::Dict, X::AbstractArray)
    intercept = ones(size(X, 1), 1)
    Z = [intercept;; X]
    
    return Z * optimal_model["optimal_coef"]
end

predict_model (generic function with 1 method)

### 3 Applying the Lasso Cross-Validation Procedure

We split the sample in train and test

In [29]:
train_sample = rand(Float64, size(data)[1]) .< 0.80
test_sample = .!(train_sample)
y_train, y_test = y[train_sample], y[test_sample]
X_flexible_train, X_flexible_test = X_flexible[train_sample, :], X_flexible[test_sample, :];

8. We perform the OLS fitting

In [30]:
using GLM, DataFrames

# Fitting a linear regression model
model_ls = lm([ones(size(X_flexible_train)[1]);;X_flexible_train],y_train)

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}:

Coefficients:
──────────────────────────────────────────────────────────────────────────────
             Coef.   Std. Error       t  Pr(>|t|)      Lower 95%     Upper 95%
──────────────────────────────────────────────────────────────────────────────
x1     4.34957        1.54363      2.82    0.0049    1.32303        7.37611
x2    -0.0685026      0.0181127   -3.78    0.0002   -0.104016      -0.0329896
x3    -0.343922       0.328121    -1.05    0.2946   -0.987257       0.299413
x4     3.16221        2.44053      1.30    0.1952   -1.62284        7.94726
x5    -1.05547        0.747389    -1.41    0.1580   -2.52085        0.409905
x6     0.117591       0.0800316    1.47    0.1418   -0.0393236      0.274506
x7    -1.71311        1.48745     -1.15    0.2495   -4.62949        1.20327
x8    -1.26601        1.48193     -0.85    0.3930   -4.17157        1.63955
x9    -

9. Now we search the optimal lambda using our `optimal_lambda` function

In [31]:
# Finding the optimal lambda and fitting the Lasso model using the training data
model_lasso = optimal_lambda(y_train, X_flexible_train, (-7, 7))

Dict{String, Any} with 4 entries:
  "optimal_coef"   => [2.88161, -0.0425397, 0.00717736, 0.0, 0.0, 0.0, -0.06902…
  "all_mse"        => [254.766, 250.117, 245.394, 241.014, 236.857, 233.738, 23…
  "optimal_lambda" => 0.0100925
  "all_lambdas"    => [0.000911882, 0.0010504, 0.00120996, 0.00139375, 0.001605…

In [32]:
# Printing the optimal lambda
model_lasso["optimal_lambda"]

0.010092531380432784

In [33]:
# Printing the optimal coefficients
model_lasso["optimal_coef"]

965-element Vector{Float64}:
  2.8816069388780243
 -0.04253968232673603
  0.007177356099954845
  0.0
  0.0
  0.0
 -0.06902284516793153
  0.0
  0.2097617552732548
  0.34980991650163623
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.26645731942236206
  0.0
  0.0

10. Now we use HDM for Julia (HDMjl) to estimate the model using the theoretically optimal penalty parameter.

In [34]:
using HDMjl

In [35]:
model_rlasso = rlasso(X_flexible_train, y_train)

Dict{String, Any} with 15 entries:
  "tss"          => 1364.7
  "dev"          => [-0.567691, -0.335889, 0.391159, -0.0143057, -0.0143057, -0…
  "model"        => [0.0 18.0 … 0.0 0.0; 1.0 25.0 … 0.0 0.0; … ; 0.0 11.0 … 0.0…
  "loadings"     => [0.237038, 5.11852, 1.94621, 7.13909, 26.7474, 0.201206, 0.…
  "sigma"        => 0.481193
  "lambda0"      => 620.327
  "lambda"       => [147.041, 3175.16, 1207.29, 4428.57, 16592.1, 124.813, 132.…
  "intercept"    => 2.84512
  "iter"         => 5
  "residuals"    => [-0.527267, -1.05785, 0.346556, -0.00863821, 0.246606, -0.2…
  "rss"          => 963.929
  "index"        => Bool[0, 1, 0, 0, 0, 1, 0, 1, 1, 0  …  0, 0, 0, 0, 0, 0, 0, …
  "beta"         => [0.0, 0.00899254, 0.0, 0.0, 0.0, -0.0765949, 0.0, 0.237952,…
  "options"      => Dict{String, Any}("intercept"=>true, "post"=>true, "meanx"=…
  "coefficients" => [2.84512, 0.0, 0.00899254, 0.0, 0.0, 0.0, -0.0765949, 0.0, …

As you may notice, the optimal penalty parameter resulting from this procedure is not comparable in size to the cross validation result. This is due to the fact that this penalty is the theoretically optimal parameter for the Lasso estimator under data-driven penalty loadings. That is:

\begin{equation*}

\hat{\beta} = \arg \ \underset{\beta}{\min} \sum_{i=1}^n (y_i - x_{i}^{\prime}\beta)^2 + \frac{\lambda}{n} \lVert \hat{\Psi}\beta \rVert_1

\end{equation*}

Where $\hat{\Psi} = diag(\hat{\psi_1},\hat{\psi_2},\dots,\hat{\psi_p})$ are the data-driven penalty loadings chosen to be a function of the data depending on the setting. For more detail, you can check the [package documentation](https://arxiv.org/pdf/1608.00354)

In [36]:
rlambda = model_rlasso["lambda0"]
rlambda

620.3272262572883

11. The predictive capability of each model (OLS, Lasso and RLasso) is reported via $MSE$ and $R^2$ out of sample

In [37]:
# OLS 

y_predict_ols = GLM.predict(model_ls, [ones(size(X_flexible_test)[1]);;X_flexible_test])
MSE_ols = mean((y_test-y_predict_ols).^2)
R2_test_ols = 1-MSE_ols/var(y_test)

0.07900145438136574

In [38]:
# Lasso CV

y_predict_lasso = predict_model(model_lasso, X_flexible_test)
MSE_lasso = mean((y_test-y_predict_lasso).^2)
R2_test_lasso = 1-MSE_lasso/var(y_test)


0.2885445136590564

In [39]:
# Rigurous Lasso
intercept = ones(size(X_flexible_test,1))
Z = [intercept;;X_flexible_test]
y_predict_rlasso = Z*model_rlasso["coefficients"]

MSE_rlasso = mean((y_test-y_predict_rlasso).^2)
R2_test_rlasso = 1-MSE_rlasso/var(y_test)

0.28903363752340694